In [1]:
from teco.models.base import *
import numpy as np
import jax
from typing import Any, Callable, Sequence
from jax import random, numpy as jnp
import flax
from flax import linen as nn

In [2]:
key1, key2 = random.split(random.PRNGKey(0))
x = random.normal(key1, (2, 64, 256, 256, 11)) # Dummy input data
model = Encoder(num_blocks=5, filters=[32, 64, 128, 256, 512], embeddings=256, num_embeddings=128)
params = model.init(key2, x, True)
out, losses = model.apply(params, x, False)
print(f"Encoder output shape: {out.shape}")

Encoder output shape: (2, 64, 3, 256)


In [4]:
key1, key2 = random.split(random.PRNGKey(0))
dummy = jnp.ones((2, 64, 3, 256))
dummy = jax.random.uniform(key=key1, shape=(2, 64, 3, 256))
aggregator = Aggregator(embed_dim=256,num_heads=16)
params_agg = aggregator.init({'params': random.PRNGKey(0), 'dropout': random.PRNGKey(1)}, out)
aggregated_variables, query = aggregator.apply(params_agg, out, rngs={'dropout': random.PRNGKey(1)})
print(f'Aggregated variables shape: {aggregated_variables.shape}')

Query shape: (128, 1, 256). X shape: (128, 3, 256)
(128, 1, 16, 16) (128, 3, 16, 16) (128, 3, 16, 16)
[ 826195449 1800638826]
Query shape: (128, 1, 256). X shape: (128, 3, 256)
(128, 1, 16, 16) (128, 3, 16, 16) (128, 3, 16, 16)
[ 826195449 1800638826]
Aggregated variables shape: (2, 64, 256)


In [5]:


key1, key2 = random.split(random.PRNGKey(0))
dummy = jnp.ones((2, 64, 3, 256))
dummy = jax.random.uniform(key=key1, shape=(2, 64, 3, 256))
deaggregator = Deaggregator(embed_dim=256,num_heads=16)
params_agg = deaggregator.init({'params': random.PRNGKey(0), 'dropout': random.PRNGKey(1)}, aggregated_variables, query)
deaggregated_variables = deaggregator.apply(params_agg, aggregated_variables,query, rngs={'dropout': random.PRNGKey(1)})
print(f'Aggregated variables shape: {deaggregated_variables.shape}')

Query shape: (128, 1, 256). X shape: (128, 3, 256)
(128, 3, 16, 16) (128, 1, 16, 16) (128, 1, 16, 16)
[ 826195449 1800638826]
Query shape: (128, 1, 256). X shape: (128, 3, 256)
(128, 3, 16, 16) (128, 1, 16, 16) (128, 1, 16, 16)
[ 826195449 1800638826]
Aggregated variables shape: (128, 3, 256)


In [6]:
decoder = Decoder(num_blocks=5, filters=[32, 64, 128, 256, 512], embeddings=256, shape=(128, 8, 8, 512))
params_decoder = decoder.init(key2, deaggregated_variables, True)
reconstruction = decoder.apply(params_decoder, deaggregated_variables, False)
print(f"Shape of the reconstruction: {reconstruction.shape}")

IndexError: tuple index out of range